In [1]:
%cd ..

/home/akajal/DiVAE


In [2]:
# Python imports
import os
import sys

# Add the path to the parent directory to augment search for module
sys.path.append(os.getcwd())
# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if par_dir not in sys.path:
    sys.path.append(par_dir)
    
# ML imports
import torch
import matplotlib.pyplot as plt
import numpy as np
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

# DiVAE imports
from models.rbm.chimeraRBM import ChimeraRBM
from models.rbm.qimeraRBM import QimeraRBM
from models.rbm.rbm import RBM
from models.samplers.pcd import PCD
from models.autoencoders.gumboltCaloCRBM import GumBoltCaloCRBM

from nbutils import *

# DWave imports
from dwave.system import DWaveSampler, LeapHybridSampler
import neal

[02:16:22.145] INFO   DiVAE                                             Willkommen!
[02:16:22.147] INFO   DiVAE                                             Loading configuration.
2022-04-07 02:16:22,946 dwave.cloud INFO  MainThread Log level for 'dwave.cloud' namespace set to 0
[02:16:22.946] INFO   dwave.cloud                                       Log level for 'dwave.cloud' namespace set to 0


In [3]:
ZERO = torch.tensor(0., dtype=torch.float)
MINUS_ONE = torch.tensor(-1., dtype=torch.float)

In [4]:
def init_model(cfg):
    model = GumBoltCaloCRBM(flat_input_size=[504],
                            train_ds_mean=0.,
                            activation_fct=torch.nn.ReLU(),
                            cfg=cfg)
    return model

In [5]:
with initialize(config_path="../configs"):
    cfg = compose(config_name="config")

/opt/conda/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [6]:
model = init_model(cfg)
model.create_networks()

2022-04-07 02:16:23,394 dwave.cloud.client.base INFO  MainThread Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
[02:16:23.394] INFO   dwave.cloud.client.base                           Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
2022-04-07 02:16:23,461 dwave.cloud.client.base INFO  MainThread Requested a solver that best matches feature filters={}
[02:16:23.461] INFO   dwave.cloud.client.base                           Requested a solver that best matches feature filters={}
2022-04-07 02:16:23,463 dwave.cloud.client.base INFO  MainThread Fetching solvers according to filters={'topology__type': 'chimera', 'chip_id': 

Initializing NetworkV3


In [7]:
run_path = '/home/akajal/DiVAE/outputs/2022-03-11/15-21-04/wandb/run-20220311_152104-2junzvyl/files/GumBoltCaloCRBM_calo_default_latest.pth'
load_state(model, run_path, 'cuda:{0}'.format(cfg.gpu_list[0]))

Loading weights for module =  _activation_fct
Loading weights for module =  _bce_loss
Loading weights for module =  _energy_activation_fct
Loading weights for module =  _hit_activation_fct
Loading weights for module =  _output_loss
Loading weights for module =  _hit_loss
Loading weights for module =  _hit_smoothing_dist_mod
Loading weights for module =  encoder
Loading weights for module =  prior
Loading weights for module =  decoder
Loading weights for module =  sampler


In [ ]:
def beta_classical(num_iterations=10, lr=0.001, beta_init=10., crbm_sampler=None, aux_crbm_sampler=None):
    assert crbm_sampler is not None
    assert aux_crbm_sampler is not None
    
    beta = beta_init
    betas = [beta]
    
    # Fixed RBM parameters
    crbm = crbm_sampler.rbm
    crbm_weights, crbm_vbias, crbm_hbias = crbm.weights, crbm.visible_bias, crbm.hidden_bias
    crbm_edgelist = crbm.pruned_edge_list
    
    # Fixed Ising parameters
    ising_weights = crbm_weights/4.
    ising_vbias = crbm_vbias/2. + torch.sum(crbm_weights, dim=1)/4.
    ising_hbias = crbm_hbias/2. + torch.sum(crbm_weights, dim=0)/4.
    
    # Sample from the fixed RBM
    crbm_vis, crbm_hid = crbm_sampler.block_gibbs_sampling()
        
    # Transform fixed RBM samples into Ising samples
    crbm_vis = torch.where(crbm_vis == ZERO, MINUS_ONE, crbm_vis)
    crbm_hid = torch.where(crbm_hid == ZERO, MINUS_ONE, crbm_hid)
        
    # Energy expectation value
    crbm_energy_exp = ising_energies_exp(ising_weights, ising_vbias, ising_hbias, crbm_vis, crbm_hid)
    crbm_energy_exp = torch.mean(crbm_energy_exp, axis=0)
    
    print("crbm_energy_exp : {0}".format(crbm_energy_exp))
    
    # Auxiliary scaled RBM
    aux_crbm = aux_crbm_sampler.rbm
    
    for i in range(num_iterations):
        
        # Calculate the rescaled parameters of the auxiliary RBM
        aux_crbm.weights = beta*crbm.weights
        aux_crbm.visible_bias = beta*crbm.visible_bias
        aux_crbm.hidden_bias = beta*crbm.hidden_bias
        aux_crbm_sampler.rbm = aux_crbm
        
        aux_crbm_vis, aux_crbm_hid = aux_crbm_sampler.block_gibbs_sampling()
        
        aux_crbm_vis = torch.where(aux_crbm_vis == ZERO, MINUS_ONE, aux_crbm_vis)
        aux_crbm_hid = torch.where(aux_crbm_hid == ZERO, MINUS_ONE, aux_crbm_hid)
        
        aux_crbm_energy_exp = ising_energies_exp(ising_weights, ising_vbias, ising_hbias, aux_crbm_vis, aux_crbm_hid)
        aux_crbm_energy_exp = torch.mean(aux_crbm_energy_exp, axis=0)
        
        print("aux_crbm_energy_exp : {0}, beta : {1}".format(aux_crbm_energy_exp, beta))
        beta = beta + lr*(-float(aux_crbm_energy_exp)+float(crbm_energy_exp))
        betas.append(beta)
        
    return betas

In [ ]:
aux_crbm = QimeraRBM(n_visible=model.prior._n_visible, n_hidden=model.prior._n_hidden)
aux_crbm_sampler = PCD(batch_size=1024, RBM=aux_crbm, n_gibbs_sampling_steps=100)
crbm_sampler = PCD(batch_size=1024, RBM=model.prior, n_gibbs_sampling_steps=100)

In [ ]:
betas = beta_classical(num_iterations=30, lr=0.005, beta_init=5., crbm_sampler=crbm_sampler, aux_crbm_sampler=aux_crbm_sampler)

In [ ]:
plot_betas(betas)

In [ ]:
def beta_qpu(num_iterations=10, lr=0.01, beta_init=10., n_reads=100, crbm_sampler=None, qpu_sampler=None):
    assert crbm_sampler is not None
    assert qpu_sampler is not None
    
    beta = beta_init
    betas = [beta]
    
    # Fixed RBM parameters
    crbm = crbm_sampler.rbm
    crbm_weights, crbm_vbias, crbm_hbias = crbm.weights, crbm.visible_bias, crbm.hidden_bias
    crbm_edgelist = crbm.pruned_edge_list
    
    # Fixed Ising parameters
    ising_weights = crbm_weights/4.
    ising_vbias = crbm_vbias/2. + torch.sum(crbm_weights, dim=1)/4.
    ising_hbias = crbm_hbias/2. + torch.sum(crbm_weights, dim=0)/4.
    
    # Sample from the fixed RBM
    crbm_vis, crbm_hid = crbm_sampler.block_gibbs_sampling()
        
    # Transform fixed RBM samples into Ising samples
    crbm_vis = torch.where(crbm_vis == ZERO, MINUS_ONE, crbm_vis)
    crbm_hid = torch.where(crbm_hid == ZERO, MINUS_ONE, crbm_hid)
        
    # Energy expectation value
    crbm_energy_exp = ising_energies_exp(ising_weights, ising_vbias, ising_hbias, crbm_vis, crbm_hid)
    crbm_energy_exp = torch.mean(crbm_energy_exp, axis=0)
    print("crbm_energy_exp : {0}".format(crbm_energy_exp))
    
    # QPU control variables
    n_vis = len(crbm.visible_qubit_idxs)
    qubit_idxs = crbm.visible_qubit_idxs+crbm.hidden_qubit_idxs
    
    visible_idx_map = {visible_qubit_idx:i for i, visible_qubit_idx in enumerate(crbm.visible_qubit_idxs)}
    hidden_idx_map = {hidden_qubit_idx:i for i, hidden_qubit_idx in enumerate(crbm.hidden_qubit_idxs)}
    
    for i in range(num_iterations):
        dwave_weights, dwave_vbias, dwave_hbias = -ising_weights/beta, -ising_vbias/beta, -ising_hbias/beta
        
        # Clamp the Ising parameters to the QPU control parameter range 
        dwave_weights = torch.clamp(dwave_weights, min=-1.5, max=0.5)
        dwave_vbias = torch.clamp(dwave_vbias, min=-1.5, max=1.5)
        dwave_hbias = torch.clamp(dwave_hbias, min=-1.5, max=1.5)
        
        dwave_weights_np = dwave_weights.detach().cpu().numpy()
        biases = torch.cat((dwave_vbias, dwave_hbias)).detach().cpu().numpy()
        
        print("Iteration = {0}, beta = {1} : J range = ({2}, {3}), h range = ({4}, {5})".format(i, beta, np.min(dwave_weights_np),
                                                                                                np.max(dwave_weights_np), np.min(biases),
                                                                                                np.max(biases)))
        
        h = {qubit_idx:bias for qubit_idx, bias in zip(qubit_idxs, biases)}
        J = {}
        for edge in crbm_edgelist:
            if edge[0] in crbm.visible_qubit_idxs:
                J[edge] = dwave_weights_np[visible_idx_map[edge[0]]][hidden_idx_map[edge[1]]]
            else:
                J[edge] = dwave_weights_np[visible_idx_map[edge[1]]][hidden_idx_map[edge[0]]]
        
        response = qpu_sampler.sample_ising(h, J, num_reads=n_reads, auto_scale=False)
        dwave_samples, dwave_energies = batch_dwave_samples(response)
        dwave_samples = torch.tensor(dwave_samples, dtype=torch.float)
        dwave_vis, dwave_hid = dwave_samples[:, :n_vis], dwave_samples[:, n_vis:]
        
        dwave_energy_exp = ising_energies_exp(ising_weights, ising_vbias, ising_hbias, dwave_vis, dwave_hid)
        dwave_energy_exp = torch.mean(dwave_energy_exp, axis=0)
        
        print("dwave_energy_exp : {0}, beta : {1}".format(dwave_energy_exp, beta))
        beta = beta - lr*(-float(dwave_energy_exp)+float(crbm_energy_exp))
        betas.append(beta)
        
    return betas

In [ ]:
qpu_sampler = DWaveSampler(solver={"topology__type":"chimera", "chip_id":"DW_2000Q_6"})
crbm_sampler = PCD(batch_size=1024, RBM=model.prior, n_gibbs_sampling_steps=100)

beta_qpu(num_iterations=20, lr=0.0001, beta_init=2., crbm_sampler=crbm_sampler, qpu_sampler=qpu_sampler)

In [ ]:
def beta_qpu_reverse(num_iterations=10, lr=0.01, beta_init=10., n_reads=100, crbm_sampler=None, qpu_sampler=None, aux_crbm_sampler=None):
    assert crbm_sampler is not None
    assert qpu_sampler is not None
    assert aux_crbm_sampler is not None
    
    beta = beta_init
    betas = [beta]
    
    # Fixed RBM parameters
    crbm = crbm_sampler.rbm
    crbm_weights, crbm_vbias, crbm_hbias = crbm.weights, crbm.visible_bias, crbm.hidden_bias
    crbm_edgelist = crbm.pruned_edge_list
    
    # Fixed Ising parameters
    ising_weights = crbm_weights/4.
    ising_vbias = crbm_vbias/2. + torch.sum(crbm_weights, dim=1)/4.
    ising_hbias = crbm_hbias/2. + torch.sum(crbm_weights, dim=0)/4.
    
    # Setup QPU control variables
    n_vis = len(crbm.visible_qubit_idxs)
    qubit_idxs = crbm.visible_qubit_idxs+crbm.hidden_qubit_idxs
    
    visible_idx_map = {visible_qubit_idx:i for i, visible_qubit_idx in enumerate(crbm.visible_qubit_idxs)}
    hidden_idx_map = {hidden_qubit_idx:i for i, hidden_qubit_idx in enumerate(crbm.hidden_qubit_idxs)}
    
    dwave_weights, dwave_vbias, dwave_hbias = -ising_weights, -ising_vbias, -ising_hbias
    
    # Clamp the Ising parameters to the QPU control parameter range 
    dwave_weights = torch.clamp(dwave_weights, min=-2., max=1.)
    dwave_vbias = torch.clamp(dwave_vbias, min=-2., max=2.)
    dwave_hbias = torch.clamp(dwave_hbias, min=-2., max=2.)
        
    dwave_weights_np = dwave_weights.detach().cpu().numpy()
    biases = torch.cat((dwave_vbias, dwave_hbias)).detach().cpu().numpy()
    
    print("J range = ({0}, {1}), h range = ({2}, {3})".format(np.min(dwave_weights_np),
                                                              np.max(dwave_weights_np),
                                                              np.min(biases),
                                                              np.max(biases)))
    
    h = {qubit_idx:bias for qubit_idx, bias in zip(qubit_idxs, biases)}
    J = {}
    for edge in crbm_edgelist:
        if edge[0] in crbm.visible_qubit_idxs:
            J[edge] = dwave_weights_np[visible_idx_map[edge[0]]][hidden_idx_map[edge[1]]]
        else:
            J[edge] = dwave_weights_np[visible_idx_map[edge[1]]][hidden_idx_map[edge[0]]]
                
    response = qpu_sampler.sample_ising(h, J, num_reads=n_reads, auto_scale=False)
    dwave_samples, dwave_energies = batch_dwave_samples(response)
    dwave_samples = torch.tensor(dwave_samples, dtype=torch.float)
    dwave_vis, dwave_hid = dwave_samples[:, :n_vis], dwave_samples[:, n_vis:]
        
    dwave_energy_exp = ising_energies_exp(ising_weights, ising_vbias, ising_hbias, dwave_vis, dwave_hid)
    dwave_energy_exp = torch.mean(dwave_energy_exp, axis=0)
    
    print("dwave_energy_exp : {0}".format(dwave_energy_exp))
    
    # Auxiliary scaled RBM
    aux_crbm = aux_crbm_sampler.rbm
    
    for i in range(num_iterations):
        
        # Calculate the rescaled parameters of the auxiliary RBM
        aux_crbm.weights = beta*crbm.weights
        aux_crbm.visible_bias = beta*crbm.visible_bias
        aux_crbm.hidden_bias = beta*crbm.hidden_bias
        aux_crbm_sampler.rbm = aux_crbm
        
        aux_crbm_vis, aux_crbm_hid = aux_crbm_sampler.block_gibbs_sampling()
        
        aux_crbm_vis = torch.where(aux_crbm_vis == ZERO, MINUS_ONE, aux_crbm_vis)
        aux_crbm_hid = torch.where(aux_crbm_hid == ZERO, MINUS_ONE, aux_crbm_hid)
        
        aux_crbm_energy_exp = ising_energies_exp(ising_weights, ising_vbias, ising_hbias, aux_crbm_vis, aux_crbm_hid)
        aux_crbm_energy_exp = torch.mean(aux_crbm_energy_exp, axis=0)
        
        print("aux_crbm_energy_exp : {0}, beta : {1}".format(aux_crbm_energy_exp, beta))
        beta = beta + lr*(-float(aux_crbm_energy_exp)+float(dwave_energy_exp))
        betas.append(beta)
        
    return betas

In [ ]:
crbm_sampler = PCD(batch_size=1024, RBM=model.prior, n_gibbs_sampling_steps=100)
qpu_sampler = DWaveSampler(solver={"topology__type":"chimera", "chip_id":"DW_2000Q_6"})

aux_crbm = QimeraRBM(n_visible=model.prior._n_visible, n_hidden=model.prior._n_hidden)
aux_crbm_sampler = PCD(batch_size=1024, RBM=aux_crbm, n_gibbs_sampling_steps=100)

In [ ]:
betas = beta_qpu_reverse(num_iterations=10,
                         lr=0.001,
                         beta_init=10.,
                         n_reads=100,
                         crbm_sampler=crbm_sampler,
                         qpu_sampler=qpu_sampler,
                         aux_crbm_sampler=aux_crbm_sampler)

In [8]:
def beta_fixed_qpu(num_iterations=10, lr=0.01, beta_init=10., n_reads=100, qpu_sampler=None, aux_crbm_sampler=None):
    assert qpu_sampler is not None
    assert aux_crbm_sampler is not None
    
    beta = beta_init
    betas = [beta]
    
    # Auxiliary scaled RBM
    aux_crbm = aux_crbm_sampler.rbm
    aux_crbm_edgelist = aux_crbm.pruned_edge_list
    
    # Setup QPU control variables
    n_vis = len(aux_crbm.visible_qubit_idxs)
    n_hid = len(aux_crbm.visible_qubit_idxs)
    qubit_idxs = aux_crbm.visible_qubit_idxs+aux_crbm.hidden_qubit_idxs
    
    visible_idx_map = {visible_qubit_idx:i for i, visible_qubit_idx in enumerate(aux_crbm.visible_qubit_idxs)}
    hidden_idx_map = {hidden_qubit_idx:i for i, hidden_qubit_idx in enumerate(aux_crbm.hidden_qubit_idxs)}
        
    ising_weights = torch.nn.Parameter(-2*torch.rand(n_vis, n_hid) + 1.5, requires_grad=False)
    #ising_weights = torch.nn.Parameter(-2*torch.rand(n_vis, n_hid) + 0.5, requires_grad=False)
    ising_vbias = torch.nn.Parameter(torch.zeros(n_vis), requires_grad=False)
    ising_hbias = torch.nn.Parameter(torch.zeros(n_hid), requires_grad=False)
    
    dwave_weights_np = -ising_weights.detach().cpu().numpy()
    print("J range = ({0}, {1})".format(np.min(dwave_weights_np),
                                        np.max(dwave_weights_np)))
    
    h = {qubit_idx:0. for qubit_idx in qubit_idxs}
    J = {}
    for edge in aux_crbm_edgelist:
        if edge[0] in aux_crbm.visible_qubit_idxs:
            J[edge] = dwave_weights_np[visible_idx_map[edge[0]]][hidden_idx_map[edge[1]]]
        else:
            J[edge] = dwave_weights_np[visible_idx_map[edge[1]]][hidden_idx_map[edge[0]]]
                
    response = qpu_sampler.sample_ising(h, J, num_reads=n_reads, auto_scale=False)
    dwave_samples, dwave_energies = batch_dwave_samples(response)
    dwave_samples = torch.tensor(dwave_samples, dtype=torch.float)
    dwave_vis, dwave_hid = dwave_samples[:, :n_vis], dwave_samples[:, n_vis:]
        
    dwave_energy_exp = ising_energies_exp(ising_weights, ising_vbias, ising_hbias, dwave_vis, dwave_hid)
    dwave_energy_exp = torch.mean(dwave_energy_exp, axis=0)
    print("dwave_energy_exp : {0}".format(dwave_energy_exp))
    
    aux_crbm._visible_bias = ising_vbias
    aux_crbm._hidden_bias = ising_hbias
    
    aux_crbm_weights = 4.*ising_weights
    
    for i in range(num_iterations):
        # Calculate the rescaled parameters of the auxiliary RBM
        aux_crbm.weights = (aux_crbm_weights*aux_crbm.weights_mask)/beta
        aux_crbm_sampler.rbm = aux_crbm
        
        aux_crbm_vis, aux_crbm_hid = aux_crbm_sampler.block_gibbs_sampling()
        
        aux_crbm_vis = torch.where(aux_crbm_vis == ZERO, MINUS_ONE, aux_crbm_vis)
        aux_crbm_hid = torch.where(aux_crbm_hid == ZERO, MINUS_ONE, aux_crbm_hid)
        
        aux_crbm_energy_exp = ising_energies_exp(ising_weights, ising_vbias, ising_hbias, aux_crbm_vis, aux_crbm_hid)
        aux_crbm_energy_exp = torch.mean(aux_crbm_energy_exp, axis=0)
        
        print("aux_crbm_energy_exp : {0}, beta : {1}".format(aux_crbm_energy_exp, beta))
        beta = beta - lr*(-float(aux_crbm_energy_exp)+float(dwave_energy_exp))
        betas.append(beta)
        
    return betas

In [9]:
qpu_sampler = DWaveSampler(solver={"topology__type":"chimera", "chip_id":"DW_2000Q_6"})

aux_crbm = QimeraRBM(n_visible=model.prior._n_visible, n_hidden=model.prior._n_hidden)
aux_crbm_sampler = PCD(batch_size=1024, RBM=aux_crbm, n_gibbs_sampling_steps=100)

2022-04-07 02:16:53,848 dwave.cloud.client.base INFO  MainThread Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
[02:16:53.848] INFO   dwave.cloud.client.base                           Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
2022-04-07 02:16:53,964 dwave.cloud.client.base INFO  MainThread Requested a solver that best matches feature filters={}
[02:16:53.964] INFO   dwave.cloud.client.base                           Requested a solver that best matches feature filters={}
2022-04-07 02:16:53,968 dwave.cloud.client.base INFO  MainThread Fetching solvers according to filters={'topology__type': 'chimera', 'chip_id': 

In [ ]:
betas = beta_fixed_qpu(num_iterations=2,
                       lr=0.1,
                       beta_init=10.,
                       n_reads=100,
                       qpu_sampler=qpu_sampler,
                       aux_crbm_sampler=aux_crbm_sampler)

In [ ]:
betas = beta_fixed_qpu(num_iterations=3,
                       lr=0.01,
                       beta_init=0.5,
                       n_reads=100,
                       qpu_sampler=qpu_sampler,
                       aux_crbm_sampler=aux_crbm_sampler)

In [13]:
betas = beta_fixed_qpu(num_iterations=10,
                       lr=0.00001,
                       beta_init=8.,
                       n_reads=100,
                       qpu_sampler=qpu_sampler,
                       aux_crbm_sampler=aux_crbm_sampler)

J range = (-1.499996542930603, 0.49999916553497314)
dwave_energy_exp : 205068.15625
aux_crbm_energy_exp : 118805.546875, beta : 8.0
aux_crbm_energy_exp : 146097.546875, beta : 7.13737390625
aux_crbm_energy_exp : 169542.984375, beta : 6.547667812499999
aux_crbm_energy_exp : 184615.296875, beta : 6.1924160937499995
aux_crbm_energy_exp : 194013.46875, beta : 5.987887499999999
aux_crbm_energy_exp : 199156.90625, beta : 5.8773406249999995
aux_crbm_energy_exp : 202632.890625, beta : 5.818228124999999
aux_crbm_energy_exp : 204107.265625, beta : 5.79387546875
aux_crbm_energy_exp : 203786.21875, beta : 5.784266562499999
aux_crbm_energy_exp : 204794.265625, beta : 5.771447187499999


In [15]:
betas = beta_fixed_qpu(num_iterations=10,
                       lr=0.0001,
                       beta_init=8.,
                       n_reads=100,
                       qpu_sampler=qpu_sampler,
                       aux_crbm_sampler=aux_crbm_sampler)

J range = (-1.4999985694885254, 0.49999547004699707)
dwave_energy_exp : 336495.4375
aux_crbm_energy_exp : 119536.796875, beta : 8.0
aux_crbm_energy_exp : 13871.400390625, beta : -13.6958640625
aux_crbm_energy_exp : 1700.158447265625, beta : -45.9582677734375
aux_crbm_energy_exp : 565.9027709960938, beta : -79.43779567871093
aux_crbm_energy_exp : 279.5589294433594, beta : -113.03074915161132
aux_crbm_energy_exp : 165.8439483642578, beta : -146.652337008667
aux_crbm_energy_exp : 106.46853637695312, beta : -180.28529636383058
aux_crbm_energy_exp : 48.814884185791016, beta : -213.92419326019288
aux_crbm_energy_exp : 38.789981842041016, beta : -247.5688555217743
aux_crbm_energy_exp : 16.75454330444336, beta : -281.2145202735901


In [ ]:
plot_betas(betas)